In [1]:
import pandas as pd
import numpy as np
import pymongo
import json
import sys

In [2]:
uri = 'mongodb://localhost:27017'
client = pymongo.MongoClient(uri)

In [3]:
db = client['ppl_data']
results = db['results']
scores = db['scores']
standings = db['standings']
print 'counts: (results, scores, standings)'
results.estimated_document_count(), scores.estimated_document_count(), standings.estimated_document_count()

counts: (results, scores, standings)


(585, 6672, 364)

In [4]:
results.find().limit(1)[0]

{u'_id': {u'group_id': 1, u'season_id': 14, u'week_id': 1},
 u'games': [u'Supersonic', u'Mata Hari', u'Stars'],
 u'location': u'NGVL',
 u'results': [{u'player': u'Brennan Ehrman',
   u'scores': [{u'points': 2, u'score': u'170,390'},
    {u'points': 1, u'score': u'52,380'},
    {u'points': 2, u'score': u'25,260'}]},
  {u'player': u'AJ Replogle',
   u'scores': [{u'points': 5, u'score': u'184,510'},
    {u'points': 2, u'score': u'52,400'},
    {u'points': 1, u'score': u'18,130'}]},
  {u'player': u'Mike Hamberg',
   u'scores': [{u'points': 1, u'score': u'68,410'},
    {u'points': 3, u'score': u'53,490'},
    {u'points': 3, u'score': u'26,720'}]},
  {u'player': u'Donnie Halinka',
   u'scores': [{u'points': 3, u'score': u'182,010'},
    {u'points': 5, u'score': u'137,730'},
    {u'points': 5, u'score': u'32,710'}]}]}

In [5]:
scores.find().limit(1)[0]

{u'_id': ObjectId('5c5c770de0e7b26ae890c05e'),
 u'game_id': 1,
 u'game_name': u'Big Game',
 u'group_id': 1,
 u'location': u'Kickback',
 u'player': u'Scott Kowalski',
 u'points': 5,
 u'position': 0,
 u'score': u'585,980',
 u'season_id': 15,
 u'week_id': 1}

In [6]:
standings.find().limit(1)[0]

{u'_id': {u'player': u'Aleksander Kaczmarczyk', u'season_id': 15},
 u'adjusted_points': 86,
 u'average_points': 13.375,
 u'points': {u'week_1': 15,
  u'week_2': 13,
  u'week_3': 15,
  u'week_4': 10,
  u'week_5': 15,
  u'week_6': 13,
  u'week_7': 15,
  u'week_8': 11},
 u'total_points': 107,
 u'weeks_played': 8}

In [7]:
df = pd.DataFrame()
for item in scores.find(projection={'_id': False}, sort=[
    ('season_id', 1), ('game_id', 1), ('week_id', 1), ('group_id', 1), ('points', -1)
]):
    df = df.append(item, ignore_index=True)
df.head()

,game_id,game_name,group_id,location,player,points,position,score,season_id,week_id
0,1.0,Supersonic,1.0,NGVL,AJ Replogle,5.0,1.0,"184,510",14.0,1.0
1,1.0,Supersonic,1.0,NGVL,Donnie Halinka,3.0,3.0,"182,010",14.0,1.0
2,1.0,Supersonic,1.0,NGVL,Brennan Ehrman,2.0,0.0,"170,390",14.0,1.0
3,1.0,Supersonic,1.0,NGVL,Mike Hamberg,1.0,2.0,"68,410",14.0,1.0
4,1.0,Stars,2.0,NGVL,Harold Lessure,5.0,0.0,"100,910",14.0,1.0


In [8]:
df.head(20)

,game_id,game_name,group_id,location,player,points,position,score,season_id,week_id
0,1.0,Supersonic,1.0,NGVL,AJ Replogle,5.0,1.0,"184,510",14.0,1.0
1,1.0,Supersonic,1.0,NGVL,Donnie Halinka,3.0,3.0,"182,010",14.0,1.0
2,1.0,Supersonic,1.0,NGVL,Brennan Ehrman,2.0,0.0,"170,390",14.0,1.0
3,1.0,Supersonic,1.0,NGVL,Mike Hamberg,1.0,2.0,"68,410",14.0,1.0
4,1.0,Stars,2.0,NGVL,Harold Lessure,5.0,0.0,"100,910",14.0,1.0
5,1.0,Stars,2.0,NGVL,Mish Paull,3.0,1.0,"98,400",14.0,1.0
6,1.0,Stars,2.0,NGVL,Scott Reppert,2.0,2.0,"74,270",14.0,1.0
7,1.0,Stars,2.0,NGVL,Erin Kelly,1.0,3.0,"46,740",14.0,1.0
8,1.0,Embryon,3.0,NGVL,Al Thomka,5.0,0.0,"651,930",14.0,1.0
9,1.0,Embryon,3.0,NGVL,James Kunz,3.0,3.0,"159,040",14.0,1.0


In [9]:
# FIXME: bad performance
def gen_arrays(var):
    var['wins_array'] = array[int(var.name)+1:]
    var['losses_array'] = array[:int(var.name)]
    return var

grouped = df.groupby(['season_id', 'week_id', 'group_id', 'game_id'])
df2 = pd.DataFrame()
for name, group in grouped:
    array = group['player'].as_matrix()[:]
    group.reset_index(inplace=True)
    group['wins'] = np.flipud(group.index.values)
    group['losses'] = group.index.values
    group = group.apply(gen_arrays, axis=1)
    df2 = df2.append(group)
df2.head(10)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

,index,game_id,game_name,group_id,location,player,points,position,score,season_id,week_id,wins,losses,wins_array,losses_array
0,0,1.0,Supersonic,1.0,NGVL,AJ Replogle,5.0,1.0,"184,510",14.0,1.0,3,0,"[Donnie Halinka, Brennan Ehrman, Mike Hamberg]",[]
1,1,1.0,Supersonic,1.0,NGVL,Donnie Halinka,3.0,3.0,"182,010",14.0,1.0,2,1,"[Brennan Ehrman, Mike Hamberg]",[AJ Replogle]
2,2,1.0,Supersonic,1.0,NGVL,Brennan Ehrman,2.0,0.0,"170,390",14.0,1.0,1,2,[Mike Hamberg],"[AJ Replogle, Donnie Halinka]"
3,3,1.0,Supersonic,1.0,NGVL,Mike Hamberg,1.0,2.0,"68,410",14.0,1.0,0,3,[],"[AJ Replogle, Donnie Halinka, Brennan Ehrman]"
0,666,2.0,Mata Hari,1.0,NGVL,Donnie Halinka,5.0,3.0,"137,730",14.0,1.0,3,0,"[Mike Hamberg, AJ Replogle, Brennan Ehrman]",[]
1,667,2.0,Mata Hari,1.0,NGVL,Mike Hamberg,3.0,2.0,"53,490",14.0,1.0,2,1,"[AJ Replogle, Brennan Ehrman]",[Donnie Halinka]
2,668,2.0,Mata Hari,1.0,NGVL,AJ Replogle,2.0,1.0,"52,400",14.0,1.0,1,2,[Brennan Ehrman],"[Donnie Halinka, Mike Hamberg]"
3,669,2.0,Mata Hari,1.0,NGVL,Brennan Ehrman,1.0,0.0,"52,380",14.0,1.0,0,3,[],"[Donnie Halinka, Mike Hamberg, AJ Replogle]"
0,1332,3.0,Stars,1.0,NGVL,Donnie Halinka,5.0,3.0,"32,710",14.0,1.0,3,0,"[Mike Hamberg, Brennan Ehrman, AJ Replogle]",[]
1,1333,3.0,Stars,1.0,NGVL,Mike Hamberg,3.0,2.0,"26,720",14.0,1.0,2,1,"[Brennan Ehrman, AJ Replogle]",[Donnie Halinka]


In [10]:
def concat_wins(df):
    return np.concatenate(df['wins_array'].values)

def concat_losses(df):
    return np.concatenate(df['losses_array'].values)

def total_stuff(series):
    series['wins'] = len(series['wins_array'])
    series['losses'] = len(series['losses_array'])
    return series

def wp(series):
    series['wp'] = float(series['wins']) / (series['wins'] + series['losses'])
    return series

def owp(series):
    opponents = np.concatenate([series['wins_array'], series['losses_array']])
    owp_total = 0
    for opponent in opponents:
        o_wins = df3.loc[(series.name[0], opponent)]['wins_array']
        o_losses = df3.loc[(series.name[0], opponent)]['losses_array']
        o_wins = [y for y in o_wins if y != series.name[1]]
        o_losses = [y for y in o_losses if y != series.name[1]]
        owp_total += ( float(len(o_wins)) / (len(o_wins) + len(o_losses)) )
    series['owp'] = owp_total / len(opponents)
    return series

def oowp(series):
    opponents = np.concatenate([series['wins_array'], series['losses_array']])
    oowp_total = 0
    for opponent in opponents:
        oowp_total += df3.loc[(series.name[0], opponent)]['owp']
    series['oowp'] = oowp_total / len(opponents)
    return series

def sos(series):
    series['sos'] = (2 * series['owp'] + series['oowp']) / 3
    return series

def rpi(series):
    series['rpi'] = (.25 * series['wp']) + (.5 * series['owp']) + (.25 * series['oowp'])
    return series

grouped = df2.groupby(['season_id', 'player'])
# for name, group in grouped:
#     print name
#     print group
    
# records = grouped.sum().drop(['index', 'game_id', 'group_id', 'points', 'position', 'week_id'], axis=1)
win_series = grouped.apply(concat_wins)
loss_series = grouped.apply(concat_losses)
df3 = pd.concat(dict(wins_array = win_series, losses_array = loss_series), axis=1)
df3 = df3.apply(total_stuff, axis=1)
df3 = df3.apply(wp, axis=1)
df3 = df3.apply(owp, axis=1)
df3 = df3.apply(oowp, axis=1)
df3 = df3.apply(sos, axis=1)
df3 = df3.apply(rpi, axis=1)
print df3
df3.loc[(14, 'Jared Schmidt')]

                                                                       losses_array  \
season_id player                                                                      
14.0      AJ Replogle             [Donnie Halinka, Mike Hamberg, Donnie Halinka,...   
          Aaron Crittenden        [Marco Leal, Justin Burkett, Justin Burkett, J...   
          Adam Pecharka           [Scott Stephens, Tyler Sutton, Dan Mitchell, T...   
          Al Thomka               [James Kunz, Kevin Vitale, Jason Rihn, Amy L. ...   
          Aleksander Kaczmarczyk  [Sara Chambers, Emily Farah, Jason Rihn, Priya...   
          Alex Fruzynski          [Jon Replogle, Jake Kolojejchick, Jon Replogle...   
          Alexander King          [Jason Potter, Raymond Karhut, Jason Potter, R...   
          Allison Thrower         [Michael Sakowitz, Michael Sakowitz, Aleksande...   
          Ally Cox                [Raymond Giorgi, Tyler Sutton, Brian Uplinger,...   
          Amy L. Covell-Murthy    [Al Thomk

losses_array    [Owen Daly, Lyle Applbaum, Brinkley Sharpe, Mi...
wins_array      [Lyle Applbaum, Owen Daly, Brinkley Sharpe, Ow...
wins                                                           47
losses                                                         22
wp                                                       0.681159
owp                                                      0.472022
oowp                                                     0.507914
sos                                                      0.483986
rpi                                                      0.533279
Name: (14.0, Jared Schmidt), dtype: object

In [11]:
df3.loc[(15, 'Jared Schmidt')]

losses_array    [Cryss Stephens, Evan Bookbinder, Evan Bookbin...
wins_array      [Raymond Karhut, Erin Kelly, Fred Cochran, Eri...
wins                                                           29
losses                                                         37
wp                                                       0.439394
owp                                                      0.588518
oowp                                                     0.504958
sos                                                      0.560665
rpi                                                      0.530347
Name: (15.0, Jared Schmidt), dtype: object

In [12]:
df4 = df3.drop(['wins_array', 'losses_array'], axis=1)
df4.reset_index(inplace=True)
stats_json = df4.to_json(orient='records')

stats = db['stats']
stats.insert_many(json.loads(stats_json))

json.loads(stats_json)

[{u'losses': 5,
  u'oowp': 0.4996371006,
  u'owp': 0.408559277,
  u'player': u'AJ Replogle',
  u'rpi': 0.4403000247,
  u'season_id': 14.0,
  u'sos': 0.4389185515,
  u'wins': 4,
  u'wp': 0.4444444444},
 {u'losses': 19,
  u'oowp': 0.4975693519,
  u'owp': 0.4851295595,
  u'player': u'Aaron Crittenden',
  u'rpi': 0.5038618796,
  u'season_id': 14.0,
  u'sos': 0.489276157,
  u'wins': 23,
  u'wp': 0.5476190476},
 {u'losses': 16,
  u'oowp': 0.4907157799,
  u'owp': 0.4497962163,
  u'player': u'Adam Pecharka',
  u'rpi': 0.449428905,
  u'season_id': 14.0,
  u'sos': 0.4634360708,
  u'wins': 11,
  u'wp': 0.4074074074},
 {u'losses': 23,
  u'oowp': 0.512736981,
  u'owp': 0.5610238136,
  u'player': u'Al Thomka',
  u'rpi': 0.5753628187,
  u'season_id': 14.0,
  u'sos': 0.5449282028,
  u'wins': 46,
  u'wp': 0.6666666667},
 {u'losses': 16,
  u'oowp': 0.5184639764,
  u'owp': 0.4845580679,
  u'player': u'Aleksander Kaczmarczyk',
  u'rpi': 0.5612889675,
  u'season_id': 14.0,
  u'sos': 0.4958600374,
  u'wins'